In [1]:
#!pip install kagglehub

GET AND CHECK DATA

In [2]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("darshue/extracted-images")

# print("Path to dataset files:", path)

IMPORTS AND DEVICE

In [3]:
import torch
import os
import os.path
import numpy as np
import sys
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import tqdm
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision.datasets as datasets

np.random.seed(111)
torch.manual_seed(111)
torch.cuda.manual_seed_all(111)

In [4]:
if torch.cuda.is_available():
    print("CUDA available")
    device = torch.device("cuda")
else:
    print("CUDA not available")
    device = torch.device("cpu")
print("Device: ", device)

CUDA available
Device:  cuda


In [5]:
train_test_split = 0.8
batch_size = 32
num_epochs = 10
learning_rate = 0.001
num_classes = 5

num_workers = 1

data_dir = '../data/train/'

In [6]:
mean, std = [0.7276], [0.1001]

In [ ]:
train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((496, 496)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((496, 496)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [8]:
data_dir = '../data/train/'

train_dataset = datasets.ImageFolder(root=os.path.join(data_dir), transform=train_transform)
test_dataset = datasets.ImageFolder(root=os.path.join(data_dir), transform=test_transform)

train_size = int(train_test_split * len(train_dataset))
test_size = len(train_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(train_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

print('Number of training samples:', len(train_loader.dataset))
print('Number of testing samples:', len(test_loader.dataset))

classes = [ 'CC', 'EC', 'HGSC', 'LGSC', 'MC' ]

Number of training samples: 20402
Number of testing samples: 5101


In [9]:
# Used to get mean and std of dataset, does not need to be run again
mean_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((496, 496)),
    transforms.ToTensor()
])
if False:
    dataset = datasets.ImageFolder(root=os.path.join(data_dir), transform=mean_transform)

    loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=num_workers)

    mean = 0.0
    std = 0.0
    nb_samples = 0

    for images, _ in loader:
        batch_samples = images.size(0)
        # Flatten images in each batch: (batch_size, channels, height * width)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        nb_samples += batch_samples

    mean /= nb_samples
    std /= nb_samples

    print('Mean:', mean)
    print('Std:', std)

In [10]:
from torchvision.models import resnet18

model = resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model = model.to(device)

c:\Users\Jordan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Jordan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.00025)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    loop = tqdm.tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    epoch_loss = running_loss / len(train_loader)

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        val_loop = tqdm.tqdm(test_loader, desc="Validation")
        for images, labels in val_loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            predictions = outputs.argmax(dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    val_accuracy = correct / total

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

Validation: 100%|██████████| 160/160 [00:24<00:00,  6.53it/s]


Epoch 1/10, Loss: 0.8516, Validation Accuracy: 0.6650


Validation: 100%|██████████| 160/160 [00:24<00:00,  6.50it/s]


Epoch 2/10, Loss: 0.6026, Validation Accuracy: 0.7152


Epoch [3/10]:  94%|█████████▍| 601/638 [02:57<00:10,  3.38it/s, loss=0.753]